# Model Prototyping

Building the basis for our model experimentation

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
import torch
import os
import json

from torch.utils import data
from torch.nn import Conv2d, AvgPool2d, ReLU, Dropout, Flatten, Linear, Sequential, Module
from torch.optim import Adam
from time import time

from tqdm import tqdm

device = torch.device("cuda:0")
torch.set_default_dtype(torch.float64)

MODELS_DIR  = '/home/cxw/sonos_rirs/models/'

In [2]:
model_dict = {}
model_dict['name'] = "testrun2_regularization"
model_dict['notes'] = "same as test run but with regularization"
model_dict['data_path'] = '/home/cxw/sonos_rirs/features/080122_5k_phase/feature_df.csv'
model_dict['model_path'] = os.path.join(MODELS_DIR, model_dict['name'])

In [3]:
feat_df = pd.read_csv(model_dict['data_path'])
model_path = os.path.join(MODELS_DIR, model_dict['name'])

dataset = []

    
# def create_dataloader(feature_df, batch_size=1, log=True):
#     dataset = []
#     for row in tqdm(feature_df.iterrows()):
#         feat_file = row[1]['file_feature']
#         loaded = np.load(feat_file)

#         feature = loaded['feat']
#         feature = feature.reshape((1, feature.shape[0], feature.shape[1]))
#         feature = np.real(feature)

#         vol = loaded['log_vol']
#         rt60 = loaded['log_rt60']
# #         if log:
# #             vol = np.log10(vol)
            
#         dataset.append((feature, vol))
#         dataset.append((feature, rt60))
    
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
#     return dataloader

# dataloader = create_dataloader(feat_df)

In [4]:
feat_df.iloc[0]['file_feature']

'/home/cxw/sonos_rirs/features/080122_5k_phase/clip0_snr_30db.npz'

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, feature_df, log=True):
        self.feature_df = feature_df
        self.log = log

    def __len__(self):
        return len(self.feature_df)

    def __getitem__(self, index):
        feat_file = self.feature_df.iloc[index]['file_feature']
#         print(feat_file)
        loaded = np.load(feat_file)

        feature = loaded['feat']
        feature = feature.reshape((feature.shape[0], feature.shape[1]))
        feature = np.real(feature)

        vol = loaded['log_vol']
        rt60 = loaded['log_rt60']

        return feature, vol, rt60


# 创建数据集实例
dataset = CustomDataset(feat_df)

# 创建数据加载器
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [6]:
for batch in dataloader:
    inputs, labels1,labels2 = batch
    # 打印每个批次的输入数据和标签
#     print("Batch Inputs:", inputs)
#     print("Labels1:", labels1)
#     print("Labels2",labels2)

In [7]:
savename = './testmodeldict.json'
with open(savename, 'w') as f:
    json.dump(model_dict, f)
    
with open(savename) as f:
    load_dict = json.load(f)
    
for key in load_dict.keys():
    print(key, load_dict[key])

name testrun2_regularization
notes same as test run but with regularization
data_path /home/cxw/sonos_rirs/features/080122_5k_phase/feature_df.csv
model_path /home/cxw/sonos_rirs/models/testrun2_regularization


In [8]:
train_df = feat_df[feat_df['split']=='train']
val_df = feat_df[feat_df['split']=='val']
test_df = feat_df[feat_df['split']=='test']

print("Creating training dataloader")
trainset = CustomDataset(train_df)
train_dataloader = DataLoader(trainset, batch_size=16)

print("Creating validation dataloader")
valset = CustomDataset(val_df)
val_dataloader = DataLoader(valset,batch_size=1)

print("Creating test dataloader")
testset = CustomDataset(test_df)
test_dataloader = DataLoader(testset,batch_size=1)

Creating training dataloader
Creating validation dataloader
Creating test dataloader


In [9]:
features, labels1,labels2 = next(iter(train_dataloader))
print(f"Feature batch shape: {features.size()}")
print(f"Labels batch shape: {labels1.size()}")
print(f"Labels batch shape: {labels2.size()}")

Feature batch shape: torch.Size([16, 30, 1997])
Labels batch shape: torch.Size([16])
Labels batch shape: torch.Size([16])


In [10]:
# -*- coding: utf-8 -*-
# @Time    : 7/16/21 3:12 AM
# @Author  : Yuan Gong
# @Affiliation  : Massachusetts Institute of Technology
# @Email   : yuangong@mit.edu
# @File    : ast_models.py

# the unified ast models for all pretraining/fine-tuning tasks.

import torch.nn as nn
import torch
import sys
sys.path.append("/data/sls/scratch/yuangong/aed-trans/src/models/")
sys.path.append("/data/sls/scratch/yuangong/aed-trans/src/")
from timm.models.layers import trunc_normal_
import timm
import numpy as np
from timm.models.layers import to_2tuple
from random import randrange
from matplotlib import pyplot as plt
import random

# override the timm package to relax the input shape constraint.
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

def get_sinusoid_encoding(n_position, d_hid):
    ''' Sinusoid position encoding table '''

    def get_position_angle_vec(position):
        return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

    return torch.FloatTensor(sinusoid_table).unsqueeze(0)

class ASTModel(nn.Module):
    def __init__(self, label_dim=527,
                 fshape=128, tshape=2, fstride=128, tstride=2,
                 input_fdim=128, input_tdim=1024, model_size='base',
                 pretrain_stage=True, load_pretrained_mdl_path=None):

        super(ASTModel, self).__init__()
        assert timm.__version__ == '0.4.5', 'Please use timm == 0.4.5, the code might not be compatible with newer versions.'

        # override timm input shape restriction
        timm.models.vision_transformer.PatchEmbed = PatchEmbed
     # 添加两个线性层
        self.fc_layer1 = nn.Linear(768, 768)
        self.fc_layer2 = nn.Linear(768, 768)
        self.FL1 = Flatten()
        self.FL2 = Flatten()  
        # pretrain the AST models
        if pretrain_stage == True:
            if load_pretrained_mdl_path != None:
                raise ValueError('Setting load_pretrained_mdl_path at pretraining stage is useless, pretraining is always from scratch, please change it to None.')
            if fstride != fshape or tstride != tshape:
                raise ValueError('fstride != fshape or tstride != tshape, they must be same at the pretraining stage, patch split overlapping is not supported.')

            # if AudioSet pretraining is not used (but ImageNet pretraining may still apply)
            if model_size == 'tiny':
                self.v = timm.create_model('vit_deit_tiny_distilled_patch16_224', pretrained=False)
                self.heads, self.depth = 3, 12
                self.cls_token_num = 2
            elif model_size == 'small':
                self.v = timm.create_model('vit_deit_small_distilled_patch16_224', pretrained=False)
                self.heads, self.depth = 6, 12
                self.cls_token_num = 2
            elif model_size == 'base':
                self.v = timm.create_model('vit_deit_base_distilled_patch16_384', pretrained=False)
                self.heads, self.depth = 12, 12
                self.cls_token_num = 2
            elif model_size == 'base_nokd':
                self.v = timm.create_model('vit_deit_base_patch16_384', pretrained=False)
                self.heads, self.depth = 12, 12
                self.cls_token_num = 1
            else:
                raise Exception('Model size must be one of tiny, small, base, base_nokd')

            self.original_num_patches = self.v.patch_embed.num_patches
            self.oringal_hw = int(self.original_num_patches ** 0.5)
            self.original_embedding_dim = self.v.pos_embed.shape[2]
            

            # SSL Pretraining Code
            self.softmax = nn.Softmax(dim=-1)
            self.lsoftmax = nn.LogSoftmax(dim=-1)
            self.fshape, self.tshape = fshape, tshape
            self.fstride, self.tstride = fstride, tstride
            self.input_fdim, self.input_tdim = input_fdim, input_tdim
            # this is a trick to make state_dict to track pretraining input_fdim and input_tdim and save them by using torch.save
            self.p_input_fdim, self.p_input_tdim = nn.Parameter(torch.tensor(input_fdim), requires_grad=False), nn.Parameter(torch.tensor(input_tdim), requires_grad=False)

            # masked patch classification (discriminative objective) layer
            # we use two layers for pretext task, but using a single layer has similar performance.
            # we map the output of transformer (768-dim for base models) to 256-dim patch input space, and then dot product with flattened patch input (also 256-dim) to calculate loss.
            # alternatively, you can map the output of transformer to 768-dim patch embedding space, and dot product with patch embedding. Performance-wise they are similar, but map to 256 space is more efficient.
            self.cpredlayer = nn.Sequential(nn.Linear(self.original_embedding_dim, self.original_embedding_dim), nn.ReLU(), nn.Linear(self.original_embedding_dim, 256))
            # masked patch reconstruction (generative objective) layer
            self.gpredlayer = nn.Sequential(nn.Linear(self.original_embedding_dim, self.original_embedding_dim), nn.ReLU(), nn.Linear(self.original_embedding_dim, 256))
            self.unfold = torch.nn.Unfold(kernel_size=(fshape, tshape), stride=(fstride, tstride))

            # we use learnable mask embedding (follow the BEIT paper), but using a fixed mask embedding (e.g., 0) leads to same performance.
            self.mask_embed = nn.Parameter(torch.zeros([1, 1, self.original_embedding_dim]))
            self.mask_embed = torch.nn.init.xavier_normal_(self.mask_embed)

            # get the intermediate shape
            self.p_f_dim, self.p_t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim, fshape, tshape)
            num_patches = self.p_f_dim * self.p_t_dim
            self.num_patches = num_patches
            self.v.patch_embed.num_patches = num_patches
            print('pretraining patch split stride: frequency={:d}, time={:d}'.format(fstride, tstride))
            print('pretraining patch shape: frequency={:d}, time={:d}'.format(fshape, tshape))
            print('pretraining patch array dimension: frequency={:d}, time={:d}'.format(self.p_f_dim, self.p_t_dim))
            print('pretraining number of patches={:d}'.format(num_patches))

            # the linear patch projection layer, use 1 channel for spectrogram rather than the original 3 channels for RGB images.
            new_proj = torch.nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
            self.v.patch_embed.proj = new_proj

            # use trainable positional embedding
            new_pos_embed = nn.Parameter(torch.zeros(1, self.v.patch_embed.num_patches + self.cls_token_num, self.original_embedding_dim))
            self.v.pos_embed = new_pos_embed
            trunc_normal_(self.v.pos_embed, std=.02)

        # use a pretrained models for finetuning
        elif pretrain_stage == False:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            if load_pretrained_mdl_path == None:
                raise ValueError('Please set load_pretrained_mdl_path to load a pretrained models.')
            sd = torch.load(load_pretrained_mdl_path, map_location=device)
            # get the fshape and tshape, input_fdim and input_tdim in the pretraining stage
            try:
                p_fshape, p_tshape = sd['module.v.patch_embed.proj.weight'].shape[2], sd['module.v.patch_embed.proj.weight'].shape[3]
                p_input_fdim, p_input_tdim = sd['module.p_input_fdim'].item(), sd['module.p_input_tdim'].item()
            except:
                raise  ValueError('The model loaded is not from a torch.nn.Dataparallel object. Wrap it with torch.nn.Dataparallel and try again.')

            print('now load a SSL pretrained models from ' + load_pretrained_mdl_path)
            # during pretraining, fstride=fshape and tstride=tshape because no patch overlapping is used
            # here, input_fdim and input_tdim should be that used in pretraining, not that in the fine-tuning.
            # we need to know input_fdim and input_tdim to do positional embedding cut/interpolation.
            # generally it should be better to use same input_fdim during pretraining and finetuning, but input_tdim can be safely different
            audio_model = ASTModel(fstride=p_fshape, tstride=p_tshape, fshape=p_fshape, tshape=p_tshape,
                                   input_fdim=p_input_fdim, input_tdim=p_input_tdim, pretrain_stage=True, model_size=model_size)
            audio_model = torch.nn.DataParallel(audio_model)
            audio_model.load_state_dict(sd, strict=False)

            self.v = audio_model.module.v
            self.original_embedding_dim = self.v.pos_embed.shape[2]
            self.cls_token_num = audio_model.module.cls_token_num

            # mlp head for fine-tuning
            self.mlp_head = nn.Sequential(nn.LayerNorm(self.original_embedding_dim),
                                          nn.Linear(self.original_embedding_dim, label_dim))

            f_dim, t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim, fshape, tshape)
            # patch array dimension during pretraining
            p_f_dim, p_t_dim = audio_model.module.p_f_dim, audio_model.module.p_t_dim
            num_patches = f_dim * t_dim
            p_num_patches = p_f_dim * p_t_dim
            self.v.patch_embed.num_patches = num_patches
            print('fine-tuning patch split stride: frequncey={:d}, time={:d}'.format(fstride, tstride))
            print('fine-tuning number of patches={:d}'.format(num_patches))

            # patch shape should be same for pretraining and fine-tuning
            if fshape != p_fshape or tshape != p_tshape:
                raise ValueError('The patch shape of pretraining and fine-tuning is not consistant, pretraining: f={:d}, t={:d}, finetuning: f={:d}, t={:d}'.format(p_fshape, p_tshape, fshape, tshape))

            # patch split stride generally should be different for pretraining and fine-tuning, as patch split overlapping is only used in finetuning
            # during pretraining, p_fshape = p_fstride and p_tshape = p_tstride
            if fstride != p_fshape or tstride != p_tshape:
                # initialize a new patch embedding layer with desired new stride.
                new_proj = torch.nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
                # but the weights of patch embedding layer is still got from the pretrained models
                new_proj.weight = torch.nn.Parameter(torch.sum(self.v.patch_embed.proj.weight, dim=1).unsqueeze(1))
                new_proj.bias = self.v.patch_embed.proj.bias
                self.v.patch_embed.proj = new_proj

            new_pos_embed = self.v.pos_embed[:, self.cls_token_num:, :].detach().reshape(1, p_num_patches, self.original_embedding_dim).transpose(1, 2).reshape(1, self.original_embedding_dim, p_f_dim, p_t_dim)
            # cut or interpolate the positional embedding
            if t_dim < p_t_dim:
                new_pos_embed = new_pos_embed[:, :, :, int(p_t_dim/2) - int(t_dim / 2): int(p_t_dim/2) - int(t_dim / 2) + t_dim]
            else:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(8, t_dim), mode='bilinear')
            if f_dim < p_f_dim:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(f_dim, t_dim), mode='bilinear')
                #new_pos_embed = new_pos_embed[:, :, int(p_f_dim/2) - int(f_dim / 2): int(p_f_dim/2) - int(f_dim / 2) + t_dim, :]
            else:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(f_dim, t_dim), mode='bilinear')

            new_pos_embed = new_pos_embed.reshape(1, self.original_embedding_dim, num_patches).transpose(1, 2)
            self.v.pos_embed = nn.Parameter(torch.cat([self.v.pos_embed[:, :self.cls_token_num, :].detach(), new_pos_embed], dim=1))

    # get the shape of intermediate representation.
    def get_shape(self, fstride, tstride, input_fdim, input_tdim, fshape, tshape):
        test_input = torch.randn(1, 1, input_fdim, input_tdim)
        test_proj = nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
        test_out = test_proj(test_input)
        f_dim = test_out.shape[2]
        t_dim = test_out.shape[3]
        return f_dim, t_dim

    # generate mask for 16*16 patch
    def gen_maskid_patch(self, sequence_len=512, mask_size=100, cluster=3):
        mask_id = []

        # randomize clutering factor in [3,6)
        cur_clus = randrange(cluster) + 3

        while len(list(set(mask_id))) <= mask_size:
            start_id = randrange(sequence_len)

            # this improves the efficiency, but might change the pretrained model
            # while start_id in mask_id:
            #     start_id = randrange(sequence_len)

            cur_mask = []
            for i in range(0, cur_clus):
                for j in range(0, cur_clus):
                    mask_cand = start_id + self.p_t_dim * i + j
                    if mask_cand > 0 and mask_cand < sequence_len:
                        cur_mask.append(mask_cand)
            mask_id = mask_id + cur_mask
        mask_id = list(set(mask_id))[:mask_size]
        return torch.tensor(mask_id)

    # using cluster for frame masking hurts the performance, so just use the naive random sampling
    def gen_maskid_frame(self, sequence_len=512, mask_size=100):
        mask_id = random.sample(range(0, sequence_len), mask_size)
        return torch.tensor(mask_id)

    def finetuningavgtok(self, x):
        B = x.shape[0]
        x = self.v.patch_embed(x)
        if self.cls_token_num == 2:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            dist_token = self.v.dist_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, dist_token, x), dim=1)
        else:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)

        for blk_id, blk in enumerate(self.v.blocks):
            x = blk(x)
        x = self.v.norm(x)

        # average output of all tokens except cls token(s)
        x = torch.mean(x[:, self.cls_token_num:, :], dim=1)
        pred1 = self.fc_layer1(x)
        pred2 = self.fc_layer2(x)
        pred1 = self.mlp_head(pred1)
        pred2 = self.mlp_head(pred2)
        pred1 =self.FL1(pred1)
        pred2 =self.FL2(pred2)
        
        return pred1,pred2

    def finetuningcls(self, x):
        B = x.shape[0]
        x = self.v.patch_embed(x)
        if self.cls_token_num == 2:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            dist_token = self.v.dist_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, dist_token, x), dim=1)
        else:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)

        for blk_id, blk in enumerate(self.v.blocks):
            x = blk(x)
        x = self.v.norm(x)

        # if models has two cls tokens (DEIT), average as the clip-level representation
        if self.cls_token_num == 2:
            x = (x[:, 0] + x[:, 1]) / 2
        else:
            x = x[:, 0]
        x = self.mlp_head(x)
        return x

    # masked patch pretraining with discriminative objective
    def mpc(self, x, mask_patch, cluster, show_mask=False):
        input = self.unfold(x).transpose(1, 2)
        B = x.shape[0]
        # x in shape (batch_size, sequence_len, embedding dim)
        x = self.v.patch_embed(x)

        # encode the patch
        # size 12(batch_size) * 100(#mask_patch) * 768(hidden_dim), prepare to save the true values of masked samples
        encode_samples = torch.empty((B, mask_patch, 256), device=x.device, requires_grad=False).float()
        # size 12(batch_size) * 100(#mask_patch), index of masked patches
        mask_index = torch.empty((B, mask_patch), device=x.device, requires_grad=False).long()
        # size 12(batch_size) * 512(sequence_len) * 768(hidden_dim)
        mask_dense = torch.ones([x.shape[0], x.shape[1], x.shape[2]], device=x.device)

        # for each audio clip in the batch
        for i in range(B):
            # randomly generate #mask_patch mask indexes without duplicate
            if cluster == True:
                # use this if you are masking e.g. 16*16 patches
                mask_index[i] = self.gen_maskid_patch(self.num_patches, mask_patch)
            else:
                # use this if you are masking frame, i.e., 128*2 patches
                mask_index[i] = self.gen_maskid_frame(self.num_patches, mask_patch)
            # copy the masked embeddings, note gradients are stopped in this path
            encode_samples[i] = input[i, mask_index[i], :].clone().detach()
            # mask the encode samples with 0
            mask_dense[i, mask_index[i], :] = 0

        # follow BEIT paper, mask with learnable masking embedding, but no performance diff observed compared with masking with 0s.
        mask_tokens = self.mask_embed.expand(B, x.shape[1], -1)

        # mask the patch
        x = x * mask_dense + (1-mask_dense) * mask_tokens

        # pass through the Transformer layers
        cls_tokens = self.v.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        dist_token = self.v.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)
        for blk in self.v.blocks:
            x = blk(x)
        x = self.v.norm(x)

        # prediction of the masked patch
        pred = torch.empty((B, mask_patch, 256), device=x.device).float()  # e.g. size 12*100*768
        for i in range(B):
            #  +2 for indexes because skipping the cls and dis token
            # we map the output of transformer (768-dim for base models) to 256-dim patch input space, and then dot product with flattened patch input (also 256-dim) to calculate loss.
            # alternatively, you can map the output of transformer to 768-dim patch embedding space, and dot product with patch embedding. Performance-wise they are similar, but map to 256 space is more efficient.
            pred[i] = self.cpredlayer(x[i, mask_index[i] + self.cls_token_num, :])

        # calculate the NCE loss
        nce = torch.tensor(0.0).to(x.device)
        correct = torch.tensor(0.0).to(x.device)
        for i in np.arange(0, B):
            # negative samples are from the same batch
            # 8/12/2022: has a difference with equation (1) in the ssast paper but (likely) performance-wise similar, see https://github.com/YuanGongND/ssast/issues/13
            total = torch.mm(encode_samples[i], torch.transpose(pred[i], 0, 1))  # e.g. size 100*100
            correct += torch.sum(torch.eq(torch.argmax(self.softmax(total), dim=0), torch.arange(0, mask_patch, device=x.device)))  # correct is a tensor
            nce += torch.sum(torch.diag(self.lsoftmax(total)))  # nce is a tensor
        acc = 1. * correct / (B * mask_patch)
        nce = nce / (-1. * B * mask_patch)

        # visualize the masked area, for probing test only, set show_mask = False for any training/inference.
        if show_mask == False:
            return acc, nce
        else:
            if B > 1:
                raise Exception('Currently only support single spectrogram probing test.')

            self.mask_correct = torch.nn.Parameter(torch.arange(0, mask_patch), requires_grad=False)

            pred = input.clone()  # [B, 512, 256]
            masked = input.clone()

            for i in range(B):
                result = [float(t) * 99 for t in torch.eq(torch.argmax(self.softmax(total), dim=0), self.mask_correct)]
                pred[i, mask_index[i], :] = torch.tensor(result).reshape(mask_patch, 1).expand(mask_patch, 256)
                masked[i, mask_index[i], :] = 99.0

            # print(total)
            # print(self.softmax(total))
            # print(torch.argmax(self.softmax(total), dim=0))
            # print(self.mask_correct)
            # print(torch.eq(torch.argmax(self.softmax(total), dim=0), self.mask_correct))
            # print([float(t)*99 for t in torch.eq(torch.argmax(self.softmax(total), dim=0), self.mask_correct)])

            fold = torch.nn.Fold(output_size=([self.input_fdim, self.input_tdim]), kernel_size=(self.fshape, self.tshape), stride=(self.fstride, self.tstride))
            pred = fold(pred.transpose(1, 2))
            masked = fold(masked.transpose(1, 2))

            return pred, masked

    # # masked patch pretraining with generative objective
    def mpg(self, input, mask_patch, cluster):
        B = input.shape[0]
        x = self.v.patch_embed(input)
        input = self.unfold(input).transpose(1, 2)

        # size 12(batch_size) * 100(#mask_patch), index of masked patches
        mask_index = torch.empty((B, mask_patch), device=x.device, requires_grad=False).long()
        # size 12(batch_size) * 512(sequence_len) * 768(hidden_dim)
        mask_dense = torch.ones([x.shape[0], x.shape[1], x.shape[2]], device=x.device)
        for i in range(B):
            # randomly generate #mask_patch mask indexes without duplicate
            if cluster == True:
                # use this if you are masking e.g. 16*16 patches
                mask_index[i] = self.gen_maskid_patch(self.num_patches, mask_patch)
            else:
                # use this if you are masking frame, i.e., 128*2 patches
                mask_index[i] = self.gen_maskid_frame(self.num_patches, mask_patch)
            mask_dense[i, mask_index[i], :] = 0

        mask_tokens = self.mask_embed.expand(B, x.shape[1], -1)

        # follow BEIT paper, mask with learnable masking embedding, but no performance diff observed compared with masking with 0s.
        x = x * mask_dense + (1-mask_dense) * mask_tokens

        # go through the Transformer layers
        cls_tokens = self.v.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        dist_token = self.v.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)
        for blk in self.v.blocks:
            x = blk(x)
        x = self.v.norm(x)

        pred = torch.empty((B, mask_patch, self.fshape * self.tshape), device=x.device).float()  # e.g. size 12*100*256
        target = torch.empty((B, mask_patch, self.fshape * self.tshape), device=x.device).float() # e.g. size 12*100*256

        for i in range(B):
            #  +2 for indexes because cls and dis token
            pred[i] = self.gpredlayer(x[i, mask_index[i] + self.cls_token_num, :])
            target[i] = input[i, mask_index[i], :]

        # calculate the MSE loss
        mse = torch.mean((pred - target) ** 2)

        return mse

    def forward(self, x, task, cluster=True, mask_patch=400):
        # expect input x = (batch_size, time_frame_num, frequency_bins), e.g., (12, 1024, 128)
        x = x.unsqueeze(1)
        x = x.transpose(2, 3)

        # finetuning (ft), use the mean of all token (patch) output as clip-level representation.
        # this is default for SSAST fine-tuning as during pretraining, supervision signal is given to each token, not the [cls] token
        if task == 'ft_avgtok':
            return self.finetuningavgtok(x)
        # alternatively, use the [cls] token output as clip-level representation.
        elif task == 'ft_cls':
            return self.finetuningcls(x)
        # pretraining, masked patch classification (discriminative objective)
        elif task == 'pretrain_mpc':
            return self.mpc(x, mask_patch=mask_patch, cluster=cluster)
        # pretraining, masked patch reconstruction (generative objective)
        elif task == 'pretrain_mpg':
            return self.mpg(x, mask_patch=mask_patch, cluster=cluster)
        elif task == 'visualize_mask':
            return self.mpc(x, mask_patch=mask_patch, cluster=cluster, show_mask=True)
        else:
            raise Exception('Task unrecognized.')

if __name__ == '__main__':
    

    # fine-tuning stage
    # now you have a labeled dataset you want to finetune AST on
    # suppose the avg length is 100 frames (1s) and there are 35 classes
    # the fshape and tshape must be same in pretraining and finetuning
    # but fstride and tstride can be different in pretraining and finetuning
    # using smaller strides improves the performance but also increase the computational overhead
    # set pretrain_stage as False since now is in the finetuning stage
    # provide the path of the pretrained model you want to load
    input_tdim = 1997  # fine-tuning data length can be different with pretraining data length
    ast_mdl = ASTModel(label_dim=1,
                 fshape=16, tshape=16, fstride=10, tstride=10,
                 input_fdim=30, input_tdim=input_tdim, model_size='base',
                 pretrain_stage=False, load_pretrained_mdl_path='./SSAST-Base-Patch-400.pth')
    # # alternatively, use a frame based AST model
    # ast_mdl = ASTModel(label_dim=35,
    #              fshape=128, tshape=2, fstride=128, tstride=1,
    #              input_fdim=128, input_tdim=input_tdim, model_size='base',
    #              pretrain_stage=False, load_pretrained_mdl_path='./test_mdl.pth')

    # do finetuning, see src/traintest.py for our finetuning code
#     test_input = torch.zeros([10, input_tdim, 30])
#     prediction = ast_mdl(test_input, task='ft_avgtok')
#     # output should in shape [batch_size, label_dim]
#     print(prediction.shape)
    # calculate the loss, do back propagate, etc

    # # (optional) do some probe test
    # test_input = torch.zeros([1, input_tdim, 128]).to(device)
    # acc, nce = ast_mdl(test_input, task='pretrain_mpc', mask_patch=100)
    # # you can visualize the mask
    # pred, masked = ast_mdl(test_input, task='visualize_mask', mask_patch=100)
    # plt.imshow(masked[0,0])
    # plt.show()

now load a SSL pretrained models from ./SSAST-Base-Patch-400.pth
pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
fine-tuning patch split stride: frequncey=10, time=10
fine-tuning number of patches=398


In [11]:
input_height = features.size()[1]
input_width = features.size()[2]
model = ASTModel(label_dim=1,
                 fshape=16, tshape=16, fstride=10, tstride=10,
                 input_fdim=30, input_tdim=input_tdim, model_size='base',
                 pretrain_stage=False, load_pretrained_mdl_path='./SSAST-Base-Patch-400.pth').to(device)

now load a SSL pretrained models from ./SSAST-Base-Patch-400.pth
pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
fine-tuning patch split stride: frequncey=10, time=10
fine-tuning number of patches=398


In [12]:
def MSE(output, target):
    loss = torch.mean((output - target)**2)
    return loss

def NMSE(output, target):
    variance = (target ** 2).mean()
    mse = ((output - target) ** 2).mean()
    normalized_mse = mse / variance
    return normalized_mse

def Bias(output, target):
    loss = torch.mean(output - target)
    return loss

def CovStep(output, target, output_mean, target_mean):
    loss = torch.mean(((output - output_mean) * (target - target_mean)))
    return loss

def MeanAbsLogStep(output, target, log=True):
    #convert out of log
    if log:
        vol_pred = 10**output
        vol_target = 10**target
    else:
        vol_pred = output
        vol_target = target
    loss = torch.mean(torch.abs(torch.log(vol_pred/vol_target)))
    return loss

def compute_eval_metrics(dataloader, model, log=True):
    target_sum1 = 0
    target_sum2 = 0
    pred_sum1 = 0
    pred_sum2 = 0
    n_steps = 0
    
    for (x,y,z) in dataloader:        
        (x, y, z) = (x.to(device), y.to(device), z.to(device))
       # print("x:",x)
        pred1,pred2 = model(x,task='ft_avgtok')
#         pred1 = pred.index_select(1, torch.tensor([0])) #########pred1--->vol_pred
#         pred2 = pred.index_select(1, torch.tensor([1])) #########pred2--->rt60_pred
#         pred1 = pred[0,0]
#         pred2 = pred[0,1]
        
        
        target_sum1 += y.sum()
        target_sum2 += z.sum()
        pred_sum1 += pred1.sum()
        pred_sum2 += pred2.sum()
        n_steps += 1
    
    target_mean1 = target_sum1/n_steps
    target_mean2 = target_sum2/n_steps
    pred_mean1 = pred_sum1/n_steps
    pred_mean2 = pred_sum2/n_steps
    
    mse1 = 0
    mse2 =0
    mean_error1=0
    mean_error2 = 0
    cov1 = 0
    cov2=0
    abs_log_ratio1 = 0
    abs_log_ratio2 = 0
    
    var_pred1 = 0 #technically var * N but gets cancelled out in Pearson calculation
    var_pred2 = 0 
    var_target1 = 0
    var_target2 = 0
    
    for (x,y,z) in dataloader:        
        (x, y, z) = (x.to(device), y.to(device), z.to(device))
       # print("x:",x)
        pred1,pred2 = model(x,task='ft_avgtok')
#         pred1 = pred.index_select(1, torch.tensor([0])) #########pred1--->vol_pred
#         pred2 = pred.index_select(1, torch.tensor([1])) #########pred2--->rt60_pred
#         pred1 = pred[0,0]
#         pred2 = pred[0,1]
        
        mse1 += MSE(pred1, y)
        mse2 += MSE(pred2, z)
        mean_error1 += Bias(pred1, y)
        mean_error2 += Bias(pred2, z)
        cov1 += CovStep(pred1, y, pred_mean1, target_mean1)
        cov2 += CovStep(pred2, z, pred_mean2, target_mean2)
        abs_log_ratio1 += MeanAbsLogStep(pred1, y, log=log)
        abs_log_ratio2 += MeanAbsLogStep(pred2, z, log=log)
        
        var_pred1 += MSE(pred1, pred_mean1)
        var_pred2 += MSE(pred2, pred_mean2)
        var_target1 += MSE(y, target_mean1)
        var_target2 += MSE(z, target_mean2)
        
        pears1 = CovStep(pred1, y, pred_mean1, target_mean1)/(torch.sqrt(MSE(pred1, pred_mean1))*torch.sqrt(MSE(y, target_mean1)))
        pears2 = CovStep(pred2, z, pred_mean2, target_mean2)/(torch.sqrt(MSE(pred2, pred_mean2))*torch.sqrt(MSE(z, target_mean2)))
    
    out_dict = {}
    out_dict['vol_mse'] = (mse1 / n_steps).item()
    out_dict['rt60_mse'] = (mse2 / n_steps).item()
    out_dict['vol_bias'] = (mean_error1 / n_steps).item()
    out_dict['rt60_bias'] = (mean_error2 / n_steps).item()
    out_dict['vol_pearson_cor'] = (cov1/(torch.sqrt(var_pred1) * torch.sqrt(var_target1))).item()
    out_dict['rt60_pearson_cor'] = (cov2/(torch.sqrt(var_pred2) * torch.sqrt(var_target2))).item()
    #out_dict['mean_mult'] = (torch.exp(abs_log_ratio/n_steps)).item()
    
    return out_dict
    
# eval_dict = compute_eval_metrics(val_dataloader, model)
# print(eval_dict)

In [13]:
opt = Adam(model.parameters(),lr=0.000002, weight_decay=1e-2)

def save_checkpoint(epoch, model, opt, filename='checkpoint.pth.tar'):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict()
    }
    torch.save(state, filename)

def load_checkpoint(filename='checkpoint.pth.tar'):
    checkpoint = torch.load(filename)
    return checkpoint['epoch'], checkpoint['model_state_dict'], checkpoint['optimizer_state_dict']

# 设置保存模型的文件夹和文件名
save_dir = 'model_checkpoints'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
checkpoint_path = os.path.join(save_dir, 'checkpoint.pth.tar')

start_epoch = 0
if os.path.isfile(checkpoint_path):
    print(f"Loading checkpoint from '{checkpoint_path}'")
    start_epoch, model_state_dict, optimizer_state_dict = load_checkpoint(checkpoint_path)
    model.load_state_dict(model_state_dict)
    opt.load_state_dict(optimizer_state_dict)
    print(f"Resuming from epoch {start_epoch}")

hist = {
	"train_loss": [],
	"vol_loss": [],
    "rt60_loss":[],
    "vol_bias": [],
    "rt60_bias":[],
    "vol_pearson_cor": [],
    "rt60_pearson_cor":[]
}

for ep in range(100):     #########################################################################                   
    t_start = time()
    model.train()
    
    train_loss = 0
    val_loss = 0
    train_steps = 0
    val_steps = 0
    
    for (x, y, z) in train_dataloader:
        (x, y, z) = (x.to(device), y.to(device), z.to(device))
        ###################################################################################################
        # 获取批次大小
        batch_size = x.size(0)
    
        # 计算需要随机选择的样本数量（25%）
        num_samples_to_select = batch_size // 4 # 16 // 4 = 4
    
        # 生成随机索引
        random_indices = torch.randperm(batch_size)[:num_samples_to_select]
    
        # 根据随机索引选择样本
        x_selected = x[random_indices]
        y_selected = y[random_indices]
        z_selected = z[random_indices]
        
        # x_selected 的形状是 [num_samples_to_select, channels, freq_dim, time_dim]
        _, freq_dim, time_dim = x_selected.shape
    
       
        # 设定掩蔽矩形块的数量和大小
        num_blocks = np.random.randint(8, 12)  # 每个样本掩蔽的块数
        block_freq_size = np.random.randint(3, 7)  # 频率维度的块大小
        block_time_size = np.random.randint(80, 120)  # 时间维度的块大小

        # 对 x_selected 中的每个样本进行掩蔽操作
        for i in range(num_samples_to_select):
            for _ in range(num_blocks):
                # 随机选择块的起始位置
                start_freq = np.random.randint(0, freq_dim - block_freq_size)
                start_time = np.random.randint(0, time_dim - block_time_size)
            
                # 计算块的结束位置
                end_freq = start_freq + block_freq_size
                end_time = start_time + block_time_size
            
                # 掩蔽块
                x_selected[i, start_freq:end_freq, start_time:end_time] = 0
        
        
        
        
        #####画图####
        # 通常我们会取batch中的第一个样本
        x_selected_cpu = x_selected.cpu()
        example_spectrogram = x_selected_cpu[0, :, :].numpy()

#         # 绘制语谱图
#         # 绘制语谱图
#         import matplotlib.pyplot as plt
#         plt.figure(figsize=(10, 6))
#         plt.imshow(example_spectrogram, aspect='auto', origin='lower', vmin=0, vmax=0.5)
# #         plt.colorbar(format='%+2.0f dB').ax.tick_params(labelsize=20)
#         plt.xlabel('Time', fontsize=22)
#         plt.ylabel('Frequency', fontsize=22)
#         plt.title('Feature Block', fontsize=24)
#         plt.xticks([])  # 隐藏横坐标刻度
#         plt.yticks([])  # 隐藏纵坐标刻度
# #         plt.xticks(fontsize=20)
# #         plt.yticks(fontsize=20)
        
#         plt.show()
        
        
    
        # 替换原始批次中的样本
        x[random_indices] = x_selected
        y[random_indices] = y_selected
        z[random_indices] = z_selected
       # print("x:",x)
        pred1,pred2 = model(x,task='ft_avgtok')
#         print(pred[0,0])
#         pred1 = pred.index_select(1, torch.tensor([0])) #########pred1--->vol_pred
#         pred2 = pred.index_select(1, torch.tensor([1])) #########pred2--->rt60_pred
#         pred1 = pred[0,0]
#         pred2 = pred[0,1]
#         print("pred_vol:",pred1)
#         print("pred_rt60:",pred2)
#         print("真实vol:",y)
#         print("真实rt60:",z)       
#         print("############")

        loss1 = NMSE(pred1, y.reshape((y.shape[0], 1)))
        loss2 = NMSE(pred2, z.reshape((z.shape[0], 1)))
        loss =2*loss1+1.0*loss2 #########################################loss换成normalized MSE 
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        train_loss += loss
        train_steps += 1
    
    with torch.no_grad():
        model.eval()
        
        val_metrics = compute_eval_metrics(val_dataloader, model)
    
    
    hist['train_loss'].append(train_loss/train_steps)
    hist['vol_loss'].append(val_metrics['vol_mse'])
    hist['rt60_loss'].append(val_metrics['rt60_mse'])
    hist['vol_bias'].append(val_metrics['vol_bias'])
    hist['rt60_bias'].append(val_metrics['rt60_bias'])
    hist['vol_pearson_cor'].append(val_metrics['vol_pearson_cor'])
    hist['rt60_pearson_cor'].append(val_metrics['rt60_pearson_cor'])
    
    
    t_end = time()
    
    t_elapsed = t_end - t_start
    print("Epoch: {}\tDuration: {:.2f}s\tTrain loss: {:.6f}\tVol loss: {:.6f}\tVol bias:{:.6f}\tVol Pearson correlation: {:.6f}\tRT60loss:{:.6f}\tRT60 bias:{:.6f}\tRT60 pearson correlation:{:.6f}\t"\
          .format(ep, t_elapsed, train_loss/train_steps, val_metrics['vol_mse'],\
                  val_metrics['vol_bias'], val_metrics['vol_pearson_cor'],val_metrics['rt60_mse'],val_metrics['rt60_bias'],val_metrics['rt60_pearson_cor']))
    

    if (ep + 1) % 1 == 0:
        save_checkpoint(ep + 1, model, opt, checkpoint_path)
        print(f"Checkpoint saved at epoch {ep + 1}")
    
    

Loading checkpoint from 'model_checkpoints/checkpoint.pth.tar'
Resuming from epoch 3


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import csv

# 创建一个空列表来存储pred和y的值
data_to_save = []

for (x, y, z) in test_dataloader:
    (x, y, z) = (x.to(device), y.to(device), z.to(device))
    pred1,pred2 = model(x,task='ft_avgtok')

    # 提取所有预测值
    
    

    for i in range(len(pred1)):
        data_to_save.append([pred1[i].item(), y[i].item(), pred2[i].item(), z[i].item()])

# 指定要保存的CSV文件名
csv_filename = 'predictions.csv'

# 打开CSV文件并将数据写入
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # 写入列名（如果需要）
    csv_writer.writerow(['Prediction_vol', 'Actual_vol', 'Prediction_rt60', 'Actual_rt60'])

    # 写入数据
    csv_writer.writerows(data_to_save)

print(f'Data saved to {csv_filename}')




In [ ]:
with torch.no_grad():
    eval_test = compute_eval_metrics(test_dataloader, model)
    print(eval_test)